<a href="https://colab.research.google.com/github/vmartinezarias/Curso_Ecologia_Paisaje_y-Ecoacustica/blob/main/8%20-%20Biofonias_Analisis_de_diversidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INTRODUCCIÓN**

Análisis de diversidad usando números de Hill, utilizando incidencias/frecuencias y CHAO2. Este tipo de análisis es apropiado cuando no se tienen abundancias. Como resultado se generan, Gráficas y tablas.

# 2. Generación del archivo para depósito de datos

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta para guardar el archivo Excel
file_output_path = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Labels_Format_empty.xlsx"

# Crear un nuevo archivo Excel
wb = Workbook()

# -------------------
# 1. Hoja "Vocabulario" con todos los detalles
# -------------------
ws_vocabulario = wb.active
ws_vocabulario.title = "Vocabulary"

# Datos completos del vocabulario
vocabulario_data = [
    ["Variable", "Description", "Values"],
    ["recording_file", "Exact file name, without the extension", "alfanumerico"],
    ["site", "Sampling site/folder code", "alfanumerico"],
    ["Cover_type", "Type of coverage entered manually", "Texto"],
    ["Rec_time", "Recording start time", "formato tiempo"],
    ["Label_type", "Labeling method", "Manual, software"],
    ["Labeler_LastName", "Labeler's last name(s)", "Texto"],
    ["Labeler_FirstName", "Labeler's first name(s)", "Texto"],
    ["Labeler_email", "Labeler's contact address", "alfanumerico"],
    ["Date", "Labeling date", "Day-Month-Year"],
    ["Validation", "How the label type was validated",
     "Manual(single,double), software(software-manual, f1, score, error, media, distance, sens, other)"],
    ["Validation_Value", "Corresponds to the % validation of the test performed", "% error, F1 value, other"],
    ["software", "Type of program used for labeling",
     "Raven, Audacity, Aureas, Kaleidoscope, other, auditory (when listened to)"],
    ["membership", "Degree of certainty/membership to the event given by algorithm or manual labeling uncertainty", "0-1"],
    ["sound_source", "Sound source", "Biophony, Anthropophony, Geophony"],
    ["sound_group", "Sound group present in the audio", "Applies only if it's biophony"],
    ["sound_event", "Type of group presented",
     "Biophony (Species_genus, genus_sp, morpho_n), Geophony (rain, thunder, wind, water bodies), "
     "Anthropophony (Vocabulary_anthropophony_geophony), Cover quality (forest, scrubland, grassland, "
     "high transformation, medium transformation, low transformation, high permanence, medium permanence, low permanence, other)"],
    ["t_init", "Initial time of the sound segment", "numeric in seconds"],
    ["t_end", "Final time of the sound segment", "numeric in seconds"],
    ["f_init", "Initial frequency of the sound segment", "numeric in kHz"],
    ["f_end", "Final frequency of the sound segment", "numeric in kHz"],
    ["f_min", "Maximum frequency of the sound segment", "numeric in kHz"],
    ["f_max", "Peak frequency of the sound segment", "numeric in kHz"],
    ["f_peak", "Minimum frequency of the sound segment", "numeric in kHz"],
    ["n_calls", "Number of calls throughout the file",
     "(use 0 when the specific sound sought was not found) integer numeric"],
    ["sound_type", "Type of sound",
     "Amphibians (warning call, etc), Birds (Notes, Syllables, Phrases, Song, Calls), bats (types), indet (if it cannot be assigned to any known category)"],
    ["Possition (bats)", "High, medium, low calls", None],
    ["Comments", "Labeler's comments", None]
]

# Estilos y colores específicos
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
row_colors = {
    "yellow": PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid"),
    "pink": PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid"),
    "blue": PatternFill(start_color="00BFFF", end_color="00BFFF", fill_type="solid"),
    "green": PatternFill(start_color="7CFC00", end_color="7CFC00", fill_type="solid"),
    "magenta": PatternFill(start_color="FF00FF", end_color="FF00FF", fill_type="solid")
}

# Secuencia de colores aplicada según el vocabulario
color_sequence = [
    "yellow", "yellow", "yellow", "yellow", "pink", "pink", "pink", "pink", None, None, None, None,
    "yellow", "yellow", "yellow", None, None, "blue", "blue", "blue", "blue", "blue", "blue", "green",
    None, "magenta"
]

# Aplicar datos y colores
for row_idx, row in enumerate(vocabulario_data, 1):
    for col_idx, value in enumerate(row, 1):
        cell = ws_vocabulario.cell(row=row_idx, column=col_idx, value=value)
        if row_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")
        else:
            cell.alignment = Alignment(horizontal="left")
            color = color_sequence[row_idx - 2] if row_idx - 2 < len(color_sequence) else None
            if color and color in row_colors:
                cell.fill = row_colors[color]

# -------------------
# 2. Hoja "Data" replicando colores
# -------------------
ws_datos = wb.create_sheet("Data")

# Cabeceras de datos
datos_headers = [v[0] for v in vocabulario_data[1:]]

# Aplicar cabeceras y estilos
for col_idx, header in enumerate(datos_headers, 1):
    cell = ws_datos.cell(row=1, column=col_idx, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = Alignment(horizontal="center")

# Aplicar colores en "Data" correspondientes a "Vocabulary"
for row_idx in range(2, 22):  # Asumiendo hasta 20 filas de datos
    for col_idx, header in enumerate(datos_headers, 1):
        cell = ws_datos.cell(row=row_idx, column=col_idx)
        color = color_sequence[col_idx - 1] if col_idx - 1 < len(color_sequence) else None
        if color and color in row_colors:
            cell.fill = row_colors[color]

# -------------------
# 3. Hoja "Vocabulary_anthropophony_geophony"
# -------------------
ws_vocab_antropo = wb.create_sheet("Vocabulary_anthropophony_geophony")

vocab_antropo_data = [
    ["Category", "Level_I", "Level_II", "Comments"],
    ["Geophony", "rain", None, None],
    ["Geophony", "thunder", None, None],
    ["Geophony", "wind", None, None],
    ["Geophony", "water_bodies", None, None],
    ["Anthropophony", "ground_traffic", "Car/tractor", None]
]

for row_idx, row in enumerate(vocab_antropo_data, 1):
    for col_idx, value in enumerate(row, 1):
        cell = ws_vocab_antropo.cell(row=row_idx, column=col_idx, value=value)
        if row_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")
        else:
            cell.alignment = Alignment(horizontal="left")

# -------------------
# 4. Hoja "Observations"
# -------------------
ws_observaciones = wb.create_sheet("Observations")

observaciones_data = [
    ["Explanations of matrix values"],
    ["1"],
    ["2"],
    ["3"]
]

for row_idx, row in enumerate(observaciones_data, 1):
    for col_idx, value in enumerate(row, 1):
        cell = ws_observaciones.cell(row=row_idx, column=col_idx, value=value)
        if row_idx == 1:
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = Alignment(horizontal="center")
        else:
            cell.alignment = Alignment(horizontal="left")

# Guardar el archivo Excel
wb.save(file_output_path)
print(f"Archivo Excel creado en: {file_output_path}")


# 3. Cargar datos




In [ ]:
import pandas as pd

# Cargar el archivo de Excel desde Google Drive
file_path = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/FORMATO_ETIQUETAS.xlsx"
sheet_name = "Data"

# Leer la hoja "Data" del archivo de Excel
mis_datos = pd.read_excel(file_path, sheet_name=sheet_name)
mis_datos['n_calls'] = pd.to_numeric(mis_datos['n_calls'], errors='coerce')
mis_datos.fillna('', inplace=True)  # Llenar valores NaN con cadenas vacías





# 4. Análisis y filtrado de datos




In [ ]:
import pandas as pd
import plotly.express as px

# Cargar el archivo de Excel
file_path = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/FORMATO_ETIQUETAS.xlsx"
sheet_name = "Data"
mis_datos = pd.read_excel(file_path, sheet_name=sheet_name)
mis_datos['n_calls'] = pd.to_numeric(mis_datos['n_calls'], errors='coerce')

# Opciones de filtrado - especifica las opciones aquí si deseas filtrar
cover_type_filter = None   # Reemplaza con el valor deseado o None para no filtrar
sound_source_filter = None # Reemplaza con el valor deseado o None para no filtrar
group_filter = None        # Reemplaza con el valor deseado o None para no filtrar
family_filter = None       # Reemplaza con el valor deseado o None para no filtrar
genus_filter = None        # Reemplaza con el valor deseado o None para no filtrar

# Filtrar los datos según las opciones especificadas
filtered_data = mis_datos.copy()
if cover_type_filter:
    filtered_data = filtered_data[filtered_data['Cover_type'] == cover_type_filter]
if sound_source_filter:
    filtered_data = filtered_data[filtered_data['sound_source'] == sound_source_filter]
if group_filter:
    filtered_data = filtered_data[filtered_data['group'] == group_filter]
if family_filter:
    filtered_data = filtered_data[filtered_data['Family'] == family_filter]
if genus_filter:
    filtered_data = filtered_data[filtered_data['Genus'] == genus_filter]

# Agrupar y ordenar los datos para la gráfica
plot_data = filtered_data.groupby('event')['n_calls'].sum().reset_index()
plot_data = plot_data.sort_values(by='n_calls', ascending=False).head(30)

# Crear y mostrar el gráfico con nombres inclinados y tamaño ajustado
fig = px.bar(plot_data, x='event', y='n_calls', title="Número de Llamadas por Taxón / Sonotipo", text='n_calls')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    xaxis_tickangle=-45,      # Inclina los nombres de las categorías en el eje x
    width=1200,               # Ancho del gráfico
    height=600,               # Alto del gráfico
    margin=dict(t=50, b=150)  # Ajuste de márgenes para dar más espacio a los nombres
)
fig.show()




# 5. Generar tablas para iNEXT

In [ ]:
import pandas as pd
from datetime import datetime

# Cargar el archivo de Excel
file_path = "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/FORMATO_ETIQUETAS.xlsx"
sheet_name = "Data"
mis_datos = pd.read_excel(file_path, sheet_name=sheet_name)

# Limpiar la columna recording_file y convertir a fecha
mis_datos['fecha'] = pd.to_datetime(
    mis_datos['recording_file'].str.extract(r'(\d{8}_\d{6})')[0], format="%Y%m%d_%H%M%S", errors='coerce'
)
mis_datos['dia'] = mis_datos['fecha'].dt.day

# Filtrar por group == "Aves"
datos_curva = mis_datos[mis_datos['group'] == "Aves"]

# Crear un dataframe con conteo de eventos únicos por día y por tipo de cobertura
conteo = datos_curva.groupby(['Cover_type', 'dia']).agg(event_count=('event', 'nunique')).reset_index()

# Para la categoría total, sumar todos los eventos distintos por día
conteo_total = datos_curva.groupby('dia').agg(event_count=('event', 'nunique')).reset_index()
conteo_total['Cover_type'] = 'Total'

# Unir los conteos específicos de cobertura con el conteo total
conteo = pd.concat([conteo, conteo_total], ignore_index=True)

# Reordenar el dataframe de forma descendente por la cantidad de eventos
conteo = conteo.sort_values(by='event_count', ascending=False)

# Crear los vectores para cada tipo de cobertura, incluyendo el total
listado_cover = {cover: group['event_count'].tolist() for cover, group in conteo.groupby('Cover_type')}

# Inicializar el diccionario que contendrá todos los vectores
Comp_inc_freq_ecos = {}

# Llenar el diccionario con los vectores, nombrándolos adecuadamente
for cover, counts in listado_cover.items():
    # Evitar incluir coberturas no deseadas como "l" (laguna) automáticamente
    if cover.lower() != "laguna":  # Excluir "laguna" automáticamente
        Comp_inc_freq_ecos[cover] = counts

# Mostrar el resultado final
print("Verificar a continuación, si hay uunidades de hábitat que posean menos de 2 registros")
Comp_inc_freq_ecos


6. Instalar paquetes de R

In [ ]:
# Ejecuta esto en una celda de código en Colab para instalar las librerías de R
!apt-get install -y libcurl4-openssl-dev libssl-dev libxml2-dev
!R -e "install.packages(c('iNEXT', 'ggplot2', 'openxlsx'))"


7. Llevar incidencias frecuenias a R

In [ ]:
# Cargar la biblioteca rpy2 para ejecutar código en R desde Colab
%load_ext rpy2.ipython

# Paso 1: Verificar y preparar los datos en Python
# Asegúrate de que cada elemento en `Comp_inc_freq_ecos` sea una lista de números
for habitat, data in Comp_inc_freq_ecos.items():
    print(f"{habitat}: {data[:5]}")  # Muestra los primeros valores de cada hábitat para verificar

# Convertimos `Comp_inc_freq_ecos` a un formato compatible en R
import rpy2.robjects as ro
from rpy2.robjects import ListVector

# Crear una estructura de lista en R
Comp_inc_freq_ecos_r = ListVector({k: ro.FloatVector(v) for k, v in Comp_inc_freq_ecos.items()})
ro.globalenv['Comp_inc_freq_ecos'] = Comp_inc_freq_ecos_r



# 8. Análisis inext

In [ ]:
%%R
library(iNEXT)
library(ggplot2)
library(openxlsx)
library(stringr)

# Definir el path de salida para guardar el archivo Excel en Google Drive
output_excel <- "/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Resultados/indices_diversidad.xlsx"

# Valores de q para el análisis de diversidad
q_values <- c(0, 1, 2)
type_values <- c(1, 2, 3)  # Tipos de gráficos
endpoint <- 20  # Unidades de muestreo (días)

# Crear archivo Excel para guardar resultados
wb <- createWorkbook()

# Títulos de los gráficos para cada valor de q
titles <- c("Riqueza de Especies Total: Conteo de Presencia",
            "Diversidad de Especies Ponderada por Frecuencia de Ocurrencia",
            "Dominancia de Especies Comunes en Sitios de Muestreo")

# Realizar el análisis de iNEXT para cada valor de q y generar gráficos
for (q in q_values) {
  # Realizar el análisis iNEXT
  out.incS <- iNEXT(Comp_inc_freq_ecos, q=q, datatype="incidence_freq", se=TRUE, conf=0.95, endpoint=endpoint)

  # Guardar resultados en el archivo Excel
  addWorksheet(wb, paste("Estimaciones_q", q, sep=""))
  writeData(wb, paste("Estimaciones_q", q, sep=""), out.incS$AsyEst)
  addWorksheet(wb, paste("Observaciones_q", q, sep=""))
  writeData(wb, paste("Observaciones_q", q, sep=""), out.incS$DataInfo)

  # Generar y mostrar gráficos para cada tipo de gráfico
  for (type in type_values) {
    plot_title <- titles[q + 1]

    # Generar el gráfico
    g <- ggiNEXT(out.incS, type=type, se=TRUE, facet.var="Assemblage", grey=TRUE) +
      theme_bw() +
      labs(x = "Días de muestreo", y = "Especies", title = plot_title) +
      theme(legend.position = "none") +
      theme(strip.text.x = element_text(size = 12, face="bold", color="black")) +
      theme(strip.text.x = element_text(size = 10),
            strip.background = element_rect(fill = "lightgrey", color = "black")) +
      theme(plot.margin = unit(c(1, 1, 1, 1), "cm")) +
      facet_wrap(~ str_wrap(Assemblage, width = 15), scales = "free_x", ncol = 3)  # Ajuste del texto a dos líneas

    # Mostrar el gráfico en Colab
    print(g)
  }
}

# Guardar el archivo Excel en Google Drive
saveWorkbook(wb, output_excel, overwrite = TRUE)
cat("Archivo Excel guardado en:", output_excel)

